In [152]:
# Warning control
import warnings
warnings.filterwarnings("ignore")

In [153]:
from crewai import Agent, Task, Crew, Process

## Tools

In [155]:
from crewai_tools import SerperDevTool,ScrapeWebsiteTool,WebsiteSearchTool


search_tool = SerperDevTool()  
scrape_tool = ScrapeWebsiteTool() 
flight_tool = WebsiteSearchTool(
    url="https://www.booking.com",
    search_terms="flights from your city to user's destination"
)
# map_tool = MapboxTool(api_key=os.getenv("MAPBOX_API_KEY"))  # Custom Mapbox tool to display locations on map
# geocoding_tool = MapboxGeocodingTool(api_key=os.getenv("MAPBOX_API_KEY"))  # Too

## Agents

In [160]:
tourist_searcher = Agent(
    role="Tourist Place Researcher ",
    goal="Find and summarize " 
    "the top tourist places " 
    "in Pakistan " 
    "based on user interests.",
    tools=[search_tool,scrape_tool],
    verbose=True,
    backstory=(
        "A web search expert, " 
        "proficient in finding " 
        "the most exciting tourist spots " 
        "based on the user's interests."
    )
 
)

In [161]:
flight_finder = Agent(
    role="Flight Search Expert",
    goal="Find flights to top tourist spots " 
    "in Pakistan " 
    "within the user\'s budget.",
    tools=[search_tool,scrape_tool,flight_tool], 
    verbose=True,
    memory=True,
    backstory=(
        "A dedicated travel expert " 
        "skilled in finding the best flight options " 
        "within the user's budget."
    ),
)

## Task

In [162]:
tourist_search_task = Task(
    description="Search the web " 
    "for top tourist places "
    "in {search_query} " 
    "based on the user's {interest}.",
    expected_output="A summary of the top tourist destinations " 
    "in {search_query} " 
    "based on the user's {interest} and {duration}.",
    agent=tourist_searcher
)


In [165]:
flight_search_task = Task(
    description="Search for flights " 
    "in {search_query} " 
    "based on user {budget}.",
    expected_output="List of {flight_search} flights " 
    "to major tourist destinations, " 
    "including airline names, " 
    "times, prices (within user's {budget}). ",
    agent=flight_finder
)

## Create Crew

In [166]:
crew = Crew(
    agents=[tourist_searcher,
             flight_finder],
    tasks=[tourist_search_task,
            flight_search_task],
   verbose=True 
)

2024-09-07 19:57:34,898 - 6772 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [167]:
inputs_details={
    "search_query": "Skardu", 
    "budget": "50000 PKR",
    "interest": "adventure", 
    "duration": "7 days",
    "flight_search": "Karachi to Skardu", 
}



In [168]:
result = crew.kickoff(inputs=inputs_details)

 [2024-09-07 19:57:41][DEBUG]: == Working Agent: Tourist Place Researcher 
 [2024-09-07 19:57:41][INFO]: == Starting Task: Search the web for top tourist places in Skardu based on the user's adventure.


> Entering new CrewAgentExecutor chain...


> Finished chain.
 [2024-09-07 19:57:53][DEBUG]: == [Tourist Place Researcher ] Task output: Agent stopped due to iteration limit or time limit.


 [2024-09-07 19:57:53][DEBUG]: == Working Agent: Flight Search Expert
 [2024-09-07 19:57:53][INFO]: == Starting Task: Search for flights in Skardu based on user 50000 PKR.


> Entering new CrewAgentExecutor chain...


> Finished chain.
 [2024-09-07 19:58:05][DEBUG]: == [Flight Search Expert] Task output: Agent stopped due to iteration limit or time limit.




In [169]:
from IPython.display import Markdown
Markdown(result.raw)

Agent stopped due to iteration limit or time limit.